## Model Drift Detection Preparation

This notebook is used to create inference data for the drift detection notebooks.  The plan is to run these an hour or a day before so the workshop participants can use them for their training.

This notebook will have the bare minimum necessary for the training.  The rest will be part of the N4_drift_detection.ipynb notebook to actually demonstrate using assays for drift detection.

## Steps

* Load the workspace, pipeline, and model versions.
* Perform sample inferences to:
  * Set the baseline
  * Perform "normal" inferences.
  * Perform inferences that should trigger alerts.

### Import Libraries

The first step will be to import our libraries, and set variables used through this tutorial.

In [22]:
import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

from IPython.display import display

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import datetime
import time


# ignoring warnings for demonstration
import warnings
warnings.filterwarnings('ignore')

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [23]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

### Retrieve Workspace, Pipeline, and Models

Retrieve the workspace, pipeline and model from notebook N1_deploy_a_model.ipynb.

In [24]:
## blank space to log in 

wl = wallaroo.Client()

# retrieve the previous workspace, model, and pipeline version

workspace_name = "workshop-finserv-jch"

workspace = wl.get_workspace(name=workspace_name)

# set your current workspace to the workspace that you just created
wl.set_current_workspace(workspace)

# optionally, examine your current workspace
wl.get_current_workspace()

model_name = 'classification-finserv-prime'

prime_model_version = wl.get_model(model_name)

pipeline_name = 'ccfraud-detector'

pipeline = wl.get_pipeline(pipeline_name)

display(workspace)
display(prime_model_version)
display(pipeline)


{'name': 'workshop-finserv-jch', 'id': 17, 'archived': False, 'created_by': '76b893ff-5c30-4f01-bd9e-9579a20fc4ea', 'created_at': '2024-05-02T17:29:32.171308+00:00', 'models': [{'name': 'classification-finserv-prime', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 5, 2, 17, 29, 51, 439254, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 5, 2, 17, 29, 51, 439254, tzinfo=tzutc())}, {'name': 'ccfraud-xgboost-version', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 5, 2, 17, 35, 25, 337258, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 5, 2, 17, 35, 25, 337258, tzinfo=tzutc())}], 'pipelines': [{'name': 'ccfraud-detector', 'create_time': datetime.datetime(2024, 5, 2, 17, 29, 52, 836622, tzinfo=tzutc()), 'definition': '[]'}]}

Name,classification-finserv-prime
Version,b3cfc328-5c34-4417-ba1a-6a1f57889de1
File Name,keras_ccfraud.onnx
SHA,bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-02-May 17:29:51


name,ccfraud-detector
created,2024-05-02 17:29:52.836622+00:00
last_updated,2024-05-02 19:25:05.330837+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"ef50af82-0cb6-4011-b72e-24c8203ab54f, 23aa5642-a6da-4b6c-8957-52255b062f85, f2b94285-7943-47c3-bc87-b2eedbdb29d1, 9c3b890e-75a9-4c62-bffa-206a1c63f5ae, fb8818a8-0646-4a1d-a0d4-fe6ee21a174d, 4143c5bf-b205-46c1-a5a2-25b556ed8935, 695a84ce-474c-4515-9670-f238d1ef48ad, f9e0b805-5dfc-465f-894e-4cd6a9f91d53, dabc98a0-bc83-4ab7-9cff-f9e846e1b0bd"
steps,classification-finserv-prime
published,False


### Deploy Pipeline

Deploy the pipeline with the model version.

In [25]:
pipeline.clear()
pipeline.add_model_step(prime_model_version)

deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
pipeline.deploy(deployment_config=deploy_config)

name,ccfraud-detector
created,2024-05-02 17:29:52.836622+00:00
last_updated,2024-05-02 19:39:39.415767+00:00
deployed,True
arch,x86
accel,none
tags,
versions,"7bac27b7-f83c-46a3-918b-c3073a45a277, ef50af82-0cb6-4011-b72e-24c8203ab54f, 23aa5642-a6da-4b6c-8957-52255b062f85, f2b94285-7943-47c3-bc87-b2eedbdb29d1, 9c3b890e-75a9-4c62-bffa-206a1c63f5ae, fb8818a8-0646-4a1d-a0d4-fe6ee21a174d, 4143c5bf-b205-46c1-a5a2-25b556ed8935, 695a84ce-474c-4515-9670-f238d1ef48ad, f9e0b805-5dfc-465f-894e-4cd6a9f91d53, dabc98a0-bc83-4ab7-9cff-f9e846e1b0bd"
steps,classification-finserv-prime
published,False


### Generate Sample Data

Before creating the assays, we must generate data for the assays to build from.

For this example, we will:

* Perform sample inferences based on lower priced houses and use that as our baseline.
* Generate inferences from specific set of high priced houses create inference outputs that will be outside the baseline.  This is used in later steps to demonstrate baseline comparison against assay analyses.

#### Inference Results History Generation

To start the demonstration, we'll create a baseline of values from houses with small estimated prices and set that as our baseline.

We will save the beginning and end periods of our baseline data to the variables `assay_baseline_start` and `assay_baseline_end`.

In [26]:
# blank spot to split or download data

high_confidence_data = pd.read_json('../data/cc_data_high_confidence.df.json')
low_confidence_data = pd.read_json('../data/cc_data_low_confidence.df.json')


In [27]:
baseline_size = 500

# Where the baseline data will start
assay_baseline_start = datetime.datetime.now()

# These inputs will be random samples
baseline_inference_set = low_confidence_data.sample(baseline_size, replace=True).reset_index(drop=True)

# Wait 60 seconds to set this data apart from the rest
time.sleep(60)
small_results = pipeline.infer(baseline_inference_set)
display(small_results)
# Set the baseline end

assay_baseline_end = datetime.datetime.now()

,time,in.tensor,out.dense_1,anomaly.count
0,2024-05-02 19:41:17.967,"[0.5386912964, 0.0261309621, -0.2192882694, 0.7520932515, 0.1653301209, -0.1371525988, 0.2867455031, 0.0418802983, -0.29498179, 0.2261193821, 0.9221485985, -0.1334537912, -1.9095456701, 1.1832246849, 0.4640579185, -0.0946957051, -0.348198258, 0.0407109537, -0.3280158784, -0.1790256397, 0.1128919431, 0.0298874984, -0.348878973, -0.5716354011, 1.3393508261, -0.5149886975, -0.0613995912, 0.0191044622, 0.6770853253]",[0.0010486543],0
1,2024-05-02 19:41:17.967,"[1.025572877, -0.0234544825, -0.6617367439, 0.2924008576, -0.0761734518, -0.9085239517, 0.1981796906, -0.3004804852, 0.3964969937, 0.0555820426, -0.6125945418, 0.7115635105, 0.5981028171, 0.1825574759, -0.0297333542, -0.1547364787, -0.3532550176, -1.0907792241, 0.1698829039, -0.233506403, -0.3864964332, -0.9311297284, 0.5804061356, 0.0928766974, -0.5336645128, 0.3984632698, -0.1715129419, -0.1717868522, -0.4498281379]",[0.0009795725],0
2,2024-05-02 19:41:17.967,"[-0.3147343557, -0.2191273828, -0.3457355825, -0.4622750017, 0.8017892037, -1.4812195685, 0.9796573187, -0.7372435414, 0.3357403519, -0.159076511, -0.2811375586, -0.211985798, -0.3380383973, 0.7121877574, 2.5473876902, -2.5660166185, 0.8601414198, 0.1028386888, 4.5709848752, -0.2460597141, 0.066864679, 1.4436926763, 0.7601480041, 0.2197738977, -0.2184374831, 0.5851737265, -0.0022475947, -0.5222059911, -0.2347711408]",[0.0016515553],0
3,2024-05-02 19:41:17.967,"[-0.1736164771, 1.1309186704, -0.5603232778, 0.5988185332, 1.2321124778, 0.6654425599, 0.247696653, -2.0682495708, -1.006045881, -0.518881723, 0.6731910547, 0.9569731785, 1.0348132894, -0.9534881945, -0.6833964392, -0.2075562248, 1.0615386332, 0.98733451, 1.284338339, -0.4122229243, 2.9873108922, -1.3872495252, -0.9867031133, -0.5172086234, 4.4791513917, -0.1139167362, 0.4025308555, 0.5213329937, -0.159997728]",[0.0017698705],0
4,2024-05-02 19:41:17.967,"[0.9927917407, 0.2448388295, -1.5234396692, 1.0527378406, 0.7317695116, -0.6752619024, 0.5559198843, -0.2457916136, 0.1765636246, -0.309647736, -0.7140641181, -0.3263373605, -0.8306122182, -0.8941113794, -0.2847943363, -0.1902398031, 1.1300115337, 0.1911989405, -0.4886621701, -0.2644643472, -0.0514342117, 0.0019234524, -0.0289688503, 0.7695968057, 0.9006320555, -1.0651341475, -0.0299424366, -0.0522389306, 0.3048854672]",[0.0014506578],0
...,...,...,...,...
495,2024-05-02 19:41:17.967,"[-0.2390153867, 0.302326788, 0.9719659516, 0.7856032169, -0.102773521, -0.5999009467, 0.1864532786, -0.05994459, -0.4055427439, 0.1034343922, -0.4712296895, -0.8969351844, -1.1298087999, 0.5004794849, 2.1453718572, -0.5181641954, 0.3768600174, 0.3364243118, 1.918316502, 0.1880934094, 0.1329553361, 0.2206871149, -0.3339438639, 0.672424471, 0.5362469185, 1.4651931021, -0.5045401651, 0.1049654592, 0.0983499788]",[0.0015901327],0
496,2024-05-02 19:41:17.967,"[-0.4462571641, 0.7059354281, -0.0792275299, -0.5247654341, 0.2276716202, -0.8602698698, 0.6977165763, -0.0951782692, 0.7183728494, 0.6020216344, -1.0360493017, -0.5300030419, -1.2762534668, 0.1223355989, -0.1585616848, -0.1495113112, -0.38931507, -0.8283441614, -0.2266710791, 0.1595220122, -0.5430712188, -1.006781274, 0.443262917, -0.2376982304, -0.6719448293, 0.3376501254, 1.1542664047, 0.8249440095, -0.8378735146]",[0.0002758503],0
497,2024-05-02 19:41:17.967,"[1.0363845785, -0.0385165042, -0.7061693318, 0.2664505727, -0.0505362128, -0.85113019, 0.1598126245, -0.2657675119, 0.4789394098, 0.0719829088, -0.8370157081, 0.3806730358, 0.0904375073, 0.2794333416, 0.0459458121, -0.0765855771, -0.3835309826, -0.9545583212, 0.2499885621, -0.2871161167, -0.4139857193, -1.0274317718, 0.5668360889, -0.1942391162, -0.524034899, 0.4234381242, -0.1777944969, -0.1835649818, -1.1314557747]",[0.0012983382],0
498,2024-05-02 19:41:17.967,"[-0.9147964053, 1.0812001015, 0.2784887914, 0.608984031, -0.4235278656, -0.1324164781, -0.197871177, 1.0328022268, -0.7442437932, -0.9516414615, -1.0064701357, 0.9563582969, 0.5468892976, 0.7751

#### Generate Numpy Baseline Values

This process generates a numpy array of the inference results used as baseline data in later steps.

In [28]:
# get the numpy values

# set the results to a non-array value
small_results_baseline_df = small_results.copy()
small_results_baseline_df['dense_1']=small_results['out.dense_1'].map(lambda x: x[0])
small_results_baseline_df

# set the numpy array
small_results_baseline = small_results_baseline_df['dense_1'].to_numpy()

#### Assay Test Data

The following will generate inference data for us to test against the assay baseline.  For this, we will add in house data that generate higher house prices than the baseline data we used earlier.

This process should take 6 minutes to generate the historical data we'll later use in our assays.  We store the DateTime `assay_window_start` to determine where to start out assay analyses.

In [30]:
# Get a spread of results

# # Set the start for our assay window period.
assay_window_start = datetime.datetime.now()

time.sleep(65)
inference_size = 1000

# Get a spread of values

window_inference_set = high_confidence_data.sample(inference_size, replace=True).reset_index(drop=True)

display(pipeline.infer(window_inference_set))

time.sleep(65)

,time,in.tensor,out.dense_1,anomaly.count
0,2024-05-02 19:42:49.071,"[-16.9005562826, 11.7940857408, -21.3499835808, 4.7464526295, -17.541821152, -3.4157578868, -19.8971731141, 13.8569141997, -3.5706261316, -7.3883763237, 3.0761156597, -4.0583425541, 1.2901028658, -2.7997535285, -0.4298746003, -4.7772250529, -11.3712952021, -5.2725617426, 0.0964799029, 4.2148077309, -0.8343371068, -2.3663572805, -1.6571937513, 0.2110054895, 4.4380877069, -0.490579919, 2.3420081006, 1.4479793289, -1.4715524921]",[0.9999745],0
1,2024-05-02 19:42:49.071,"[-9.7167931532, 9.1749815168, -14.4507610655, 8.6538247823, -11.0399514619, 0.6602411004, -22.825524836, -9.91939572, -8.0643244934, -16.7379272666, 4.852197246, -12.5633431259, -1.0762653002, -7.5245908787, -3.2938414646, -9.6210202785, -15.6501046672, -7.0897413013, 1.7687133343, 5.04490625, -11.3656249932, 4.5987033857, 4.4777047425, 0.3170269858, -2.27319765, 0.0794467478, -10.0520585128, -2.0241080271, -1.0611984956]",[1.0],0
2,2024-05-02 19:42:49.071,"[-1.0603297501, 2.3544967095, -3.5638788326, 5.1387348926, -1.2308457019, -0.7687824608, -3.5881228109, 1.8880837663, -3.2789674274, -3.9563254554, 4.0993439118, -5.6539176395, -0.8775733373, -9.131571192, -0.6093537873, -3.7480276773, -5.0309125017, -0.8748149526, 1.9870535692, 0.7005485718, 0.9204422758, -0.1041491809, 0.3229564351, -0.7418141657, 0.0384120159, 1.0993439146, 1.2603409756, -0.1466244739, -1.4463212439]",[0.99300325],0
3,2024-05-02 19:42:49.071,"[-3.3144418806, 2.4431303967, -6.1724144943, 3.6737356364, -3.8154198185, -1.5950849337, -4.8292920964, 2.9850773386, -4.2241602384, -7.5519833648, 6.1932962861, -8.5988599079, 0.2544341359, -11.834097152, -0.3958333658, -6.0153615483, -13.5327613735, -4.2268449869, 1.1153876263, 0.1798952806, 1.3166594922, -0.6443338219, 0.2305494966, -0.5776497723, 0.7609738682, 2.2197483266, 4.0118897371, -1.234766675, 1.2847252872]",[0.9999876],0
4,2024-05-02 19:42:49.071,"[-5.4078765762, 3.903996191, -8.9852198982, 5.1287423563, -7.3732235707, -2.9462338951, -11.033238351, 5.9140189212, -5.6692411698, -12.0410529008, 6.9507916373, -12.4887954952, 1.2236942609, -14.1785651067, 1.6514667267, -12.4701897494, -22.3505048699, -8.9287548775, 4.5477499454, -0.1147899378, 3.1302070304, -0.7012850433, -0.4027528414, 0.7511917888, -0.1856307915, 0.9228208574, 0.1466560098, -1.3761806303, 0.4299709671]",[1.0],0
...,...,...,...,...
995,2024-05-02 19:42:49.071,"[-0.7489371284, 1.3893062439, -3.7477516736, 2.4144503616, -0.1106142907, -1.0737498121, -3.1504632934, 1.2081384551, -1.3328719813, -4.6042761639, 4.4385482561, -7.6876878584, 1.1683422148, -5.3296028573, -0.1983868424, -5.2942426939, -5.4928791722, -1.3254275184, 4.387227841, 0.6864338402, 0.8722859856, -0.1154091016, -0.8364338056, -0.612022188, 0.1051805487, 2.2618087452, 1.143507809, -0.3262335692, -1.6081295298]",[0.9852645],0
996,2024-05-02 19:42:49.071,"[-3.1356348771, -1.4838169823, -3.0833668024, 1.6626455715, -0.5969503582, -0.3019960859, -3.3165629662, 1.8696089903, -1.8006077803, -4.5662027398, 2.8778170681, -4.088723648, -0.4340183397, -3.5816982678, 0.4517178741, -5.7251308379, -8.982029129, -4.0279546984, 0.8926447664, 0.2472187248, 1.8289507119, 1.6895254013, -2.5555578197, -2.4714023809, -0.4500012103, 0.2333302798, 2.2119385988, -2.0418050321, 1.1568314012]",[0.95601666],0
997,2024-05-02 19:42:49.071,"[-0.7489371284, 1.3893062439, -3.7477516736, 2.4144503616, -0.1106142907, -1.0737498121, -3.1504632934, 1.2081384551, -1.3328719813, -4.6042761639, 4.4385482561, -7.6876878584, 1.1683422148, -5.3296028573, -0.1983868424, -5.2942426939, -5.4928791722, -1.3254275184, 4.387227841, 0.6864338402, 0.8722859856, -0.1154091016, -0.8364338056, -0.612022188, 0.1051805487, 2.2618087452, 1.143507809, -0.3262335692, -1.6081295298]",[0.9852645],0
998,2024-05-02 19:42:49.071,"[-1.0603297501, 2.3544967095, -3.5638788326, 5.1387348926, -1.2308457019, -0.7687824608, -3.5881228109, 1.8880837663, -3.2789674274, -3.9563254554, 4.0993439118, -5.6539176395, -0

### Undeploy Main Pipeline

With the examples and tutorial complete, we will undeploy the main pipeline and return the resources back to the Wallaroo instance.

In [31]:
pipeline.undeploy()

name,ccfraud-detector
created,2024-05-02 17:29:52.836622+00:00
last_updated,2024-05-02 19:39:39.415767+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"7bac27b7-f83c-46a3-918b-c3073a45a277, ef50af82-0cb6-4011-b72e-24c8203ab54f, 23aa5642-a6da-4b6c-8957-52255b062f85, f2b94285-7943-47c3-bc87-b2eedbdb29d1, 9c3b890e-75a9-4c62-bffa-206a1c63f5ae, fb8818a8-0646-4a1d-a0d4-fe6ee21a174d, 4143c5bf-b205-46c1-a5a2-25b556ed8935, 695a84ce-474c-4515-9670-f238d1ef48ad, f9e0b805-5dfc-465f-894e-4cd6a9f91d53, dabc98a0-bc83-4ab7-9cff-f9e846e1b0bd"
steps,classification-finserv-prime
published,False


### Store the Assay Values

We will store the following into a location configuration file:

* `small_results_baseline`:  Used to create the baseline from the numpy values from sample inferences.
* `assay_baseline_start`: When to start the baseline from the inference history.
* `assay_baseline_end`: When to end the baseline from the inference history.
* `assay_window_start`: When to start the assay window period for assay samples.

In [32]:
# skip this step if the file is already there

import numpy

numpy.save('./small_results_baseline.npy', small_results_baseline)

In [33]:
baseline_numpy = numpy.load('./small_results_baseline.npy')
baseline_numpy

array([1.0486543e-03, 9.7957250e-04, 1.6515553e-03, 1.7698705e-03,
       1.4506578e-03, 1.1940598e-03, 3.8453937e-04, 3.1602383e-04,
       7.2556734e-04, 8.6218120e-04, 2.3874640e-04, 8.2156060e-04,
       1.9270182e-04, 7.2795150e-04, 6.1669946e-04, 7.3608756e-04,
       1.5822053e-04, 1.6241968e-03, 7.0184470e-04, 3.7059188e-04,
       6.7126750e-04, 6.5499544e-04, 2.7862190e-04, 1.7445982e-03,
       2.5522710e-04, 1.7794967e-03, 8.3655120e-05, 3.3876300e-04,
       6.8780780e-04, 2.2342801e-04, 3.6254525e-04, 1.9484162e-03,
       3.6540627e-04, 4.3514670e-03, 1.1000037e-04, 5.0419570e-04,
       6.3914060e-04, 3.0297040e-04, 3.7470460e-04, 9.1803074e-04,
       8.4322690e-04, 2.7117133e-04, 1.7297268e-04, 1.0255277e-03,
       1.6617775e-04, 2.4440885e-04, 6.1669946e-04, 5.7426095e-04,
       1.1988580e-03, 8.7618830e-04, 7.9396367e-04, 3.8108230e-04,
       1.6323626e-03, 1.3339520e-04, 2.0414591e-04, 1.4910102e-03,
       1.7982721e-04, 5.8496000e-04, 2.5415420e-04, 1.9714832e

In [34]:
with open('./assay_baseline_start', 'w') as file:
    file.write(assay_baseline_start.strftime("%d-%b-%Y (%H:%M:%S.%f)"))
assay_baseline_start

datetime.datetime(2024, 5, 2, 13, 40, 17, 258091)

In [35]:
with open('./assay_baseline_end', 'w') as file:
    file.write(assay_baseline_end.strftime("%d-%b-%Y (%H:%M:%S.%f)"))
assay_baseline_end

datetime.datetime(2024, 5, 2, 13, 41, 18, 91207)

In [36]:
with open('./assay_window_start', 'w') as file:
    file.write(assay_window_start.strftime("%d-%b-%Y (%H:%M:%S.%f)"))
assay_window_start

datetime.datetime(2024, 5, 2, 13, 41, 43, 398519)

In [37]:
# read the assay baseline start datetime

with open('./assay_baseline_start', 'r') as file:
    assay_baseline_start_test = datetime.datetime.strptime(file.read(), "%d-%b-%Y (%H:%M:%S.%f)")
assay_baseline_start_test

datetime.datetime(2024, 5, 2, 13, 40, 17, 258091)

In [38]:
# read the assay baseline end datetime

with open('./assay_baseline_end', 'r') as file:
    assay_baseline_end_test = datetime.datetime.strptime(file.read(), "%d-%b-%Y (%H:%M:%S.%f)")
assay_baseline_end_test

datetime.datetime(2024, 5, 2, 13, 41, 18, 91207)

In [39]:
# read the assay window start datetime

with open('./assay_window_start', 'r') as file:
    assay_window_start_test = datetime.datetime.strptime(file.read(), "%d-%b-%Y (%H:%M:%S.%f)")
assay_window_start_test

datetime.datetime(2024, 5, 2, 13, 41, 43, 398519)